In [68]:
import pandas as pd
import jpy_tools.parseSnake2 as jps

In [69]:
configPath = '/public1/software/liuzj/scripts/pipeline/dropseqAnalysisByStarsolo/snakemake/config.yaml'
snakePath = '/public/home/liuzj/scripts/pipeline/dropseqAnalysisByStarsolo/snakemake/snakefile'

In [70]:
snakeFile = jps.SnakeFile()

In [71]:
snakeHeader = jps.SnakeHeader(snakeFile, configPath)
snakeHeader.addCode("sampleLs = list(config['samples'].keys())")
config = snakeHeader.getConfig()
snakeHeader

import pandas as pd
#configfile: "/public1/software/liuzj/scripts/pipeline/dropseqAnalysisByStarsolo/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['samples'].keys())

In [72]:
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['samples'].keys())

In [73]:
def addDirToFastq(sr):
    dir_fastq = sr.at['dir_fastq']
    path_r1 = sr.at['r1']
    ls_pathR1 = path_r1.split(',')
    ls_pathR1 = [f"{dir_fastq}/{x}" for x in ls_pathR1]
    sr.at['r1'] = ",".join(ls_pathR1)
    path_r2 = sr.at['r2']
    ls_pathR2 = path_r2.split(',')
    ls_pathR2 = [f"{dir_fastq}/{x}" for x in ls_pathR2]
    sr.at['r2'] = ",".join(ls_pathR2)
    return sr
    
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/').apply(addDirToFastq, axis=1)
df_runStarsolo

,dir_fastq,r1,r2,expectedCell,umiLength,umiStart,cbLength,cbStart,r1Length,includeIntron,starIndex,outputDir
ED1,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,5000,8,13,12,1,150,True,/public/home/liuzj/data/Araport11/STAR_INDEX,ED1/
ED2,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,5000,8,13,12,1,150,True,/public/home/liuzj/data/Araport11/STAR_INDEX,ED2/
ED3,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,5000,8,13,12,1,150,True,/public/home/liuzj/data/Araport11/STAR_INDEX,ED3/
EN1,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,5000,8,13,12,1,150,True,/public/home/liuzj/data/Araport11/STAR_INDEX,EN1/
EN2,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,5000,8,13,12,1,150,True,/public/home/liuzj/data/Araport11/STAR_INDEX,EN2/
EN3,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,/public/home/liuzj/projects/singleCell/clock/0...,5000,8,13,12,1,150,True,/public/home/liuzj/data/Araport11/STAR_INDEX,EN3/


In [77]:
runStarsolo = jps.SnakeRule(snakeFile, "runStarsolo", 32, priority=20)
runStarsolo.addCode(
    """
def addDirToFastq(sr):
    dir_fastq = sr.at['dir_fastq']
    path_r1 = sr.at['r1']
    ls_pathR1 = path_r1.split(',')
    ls_pathR1 = [f"{dir_fastq}/{x}" for x in ls_pathR1]
    sr.at['r1'] = ",".join(ls_pathR1)
    path_r2 = sr.at['r2']
    ls_pathR2 = path_r2.split(',')
    ls_pathR2 = [f"{dir_fastq}/{x}" for x in ls_pathR2]
    sr.at['r2'] = ",".join(ls_pathR2)
    return sr
    
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/').apply(addDirToFastq, axis=1)
"""
)
runStarsolo.addMetaDf(
    "df_runStarsolo",
    ["outputDir"]
)
runStarsolo.addMain('input', ['dir_fastq'])
runStarsolo.addMain('params', ['expectedCell', 'r1', 'r2', 'includeIntron', 'starIndex', 'outputDir', 'umiLength', 'umiStart', 'cbLength', 'cbStart', 'r1Length'])
runStarsolo.setShell("""
mkdir -p {params.outputDir}
cd {params.outputDir}
if [ {params.includeIntron} = True ]
then
    STAR --runThreadN {threads} --genomeDir {params.starIndex} --soloType CB_UMI_Simple --soloCBwhitelist None --soloBarcodeReadLength {params.r1Length}\
        --soloUMIlen {params.umiLength} --soloCBlen {params.cbLength} --soloUMIstart {params.umiStart} --soloCBstart {params.cbStart} --readFilesIn {params.r2} {params.r1} --soloFeatures Gene GeneFull Velocyto\
        --clipAdapterType CellRanger4 --outFilterScoreMin 30 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR\
        --soloUMIdedup 1MM_CR --soloCellFilter  EmptyDrops_CR {params.expectedCell} 0.99 10 45000 90000 500 0.01 20000 0.01 10000 \
        --readFilesCommand zcat
else
    STAR --runThreadN {threads} --genomeDir {params.starIndex} --soloType CB_UMI_Simple --soloCBwhitelist None --soloBarcodeReadLength {params.r1Length}\
        --soloUMIlen {params.umiLength} --soloCBlen {params.cbLength} --soloUMIstart {params.umiStart} --soloCBstart {params.cbStart}  --readFilesIn {params.r2} {params.r1} --soloFeatures Gene Velocyto\
        --clipAdapterType CellRanger4 --outFilterScoreMin 30 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR\
        --soloUMIdedup 1MM_CR --soloCellFilter  EmptyDrops_CR {params.expectedCell} 0.99 10 45000 90000 500 0.01 20000 0.01 10000 \
        --readFilesCommand zcat
fi
samtools view -@ {threads} -S -O BAM ./Aligned.out.sam  | samtools sort -@ {threads} - -O BAM -o ./Aligned.out.bam
samtools index -@ {threads} ./Aligned.out.bam
rm ./Aligned.out.sam
""")
runStarsolo

2022-10-20 16:37:46.065 | INFO     | jpy_tools.parseSnake2:addRule:55 - runStarsolo step num: 1
2022-10-20 16:37:46.067 | WARNING  | jpy_tools.parseSnake2:addMetaDf:203 - please set `metaDf` if you want to record dataframe content in snakefile



## get parameter of rule `runStarsolo` ##
def addDirToFastq(sr):
    dir_fastq = sr.at['dir_fastq']
    path_r1 = sr.at['r1']
    ls_pathR1 = path_r1.split(',')
    ls_pathR1 = [f"{dir_fastq}/{x}" for x in ls_pathR1]
    sr.at['r1'] = ",".join(ls_pathR1)
    path_r2 = sr.at['r2']
    ls_pathR2 = path_r2.split(',')
    ls_pathR2 = [f"{dir_fastq}/{x}" for x in ls_pathR2]
    sr.at['r2'] = ",".join(ls_pathR2)
    return sr
    
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/').apply(addDirToFastq, axis=1)
for column in ['outputDir']:
    df_runStarsolo[column] = resultDir + 'step1_runStarsolo/' + df_runStarsolo[column]
----------------
IN RULE
----------------
rule runStarsolo:
    input:
        dir_fastq = lambda wildcard: df_runStarsolo.at[wildcard.sample, 'dir_fastq'],
    output:
        runStarsoloFinished = resultDir + 'step1_runStarsolo/' + '{sample}.finished',
    param

In [78]:
snakeAll = jps.SnakeAll(snakeFile, runStarsolo)
snakeAll

rule all:
    input:
        runStarsoloFinished = [resultDir + 'step1_runStarsolo/' + "" + sample + ".finished" for sample in df_runStarsolo.index],

In [79]:
snakeFile.getMain(snakePath)

import pandas as pd
#configfile: "/public1/software/liuzj/scripts/pipeline/dropseqAnalysisByStarsolo/snakemake/config.yaml"
pipelineDir = config['pipelineDir']
resultDir = config["resultDir"].rstrip("/") + "/"
pipelineDir = config["pipelineDir"].rstrip("/") + "/"
sampleLs = list(config['samples'].keys())

## get parameter of rule `runStarsolo` ##
def addDirToFastq(sr):
    dir_fastq = sr.at['dir_fastq']
    path_r1 = sr.at['r1']
    ls_pathR1 = path_r1.split(',')
    ls_pathR1 = [f"{dir_fastq}/{x}" for x in ls_pathR1]
    sr.at['r1'] = ",".join(ls_pathR1)
    path_r2 = sr.at['r2']
    ls_pathR2 = path_r2.split(',')
    ls_pathR2 = [f"{dir_fastq}/{x}" for x in ls_pathR2]
    sr.at['r2'] = ",".join(ls_pathR2)
    return sr
    
df_runStarsolo = pd.DataFrame(config['samples']).T
df_runStarsolo = df_runStarsolo.assign(starIndex = config['starIndex'], outputDir=df_runStarsolo.index + '/').apply(addDirToFastq, axis=1)
for column in ['outputDir']:
    df_runStarsolo[column] = resultDir + 'ste